In [1]:
 # This Python 3 environment comes with many helpful analytics libraries installed
 # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
 # For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gate-dsai-llm/build_index.ipynb
/kaggle/input/gate-dsai-llm/GATE2024_DA_Sample_Paper.pdf
/kaggle/input/gate-dsai-llm/README.md
/kaggle/input/gate-dsai-llm/Chat_bot.ipynb
/kaggle/input/gate-dsai-llm/GATE_DA_2025_Question_Paper.pdf
/kaggle/input/gate-dsai-llm/GATE_DA_2025_Syllabus.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/ML-A-Probabilistic-Perspective-Murphy.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/UPenn-CIS520-Midterm2019-Solutions.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/CMU-ML-Notes.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/CS229-Stanford-Lecture-Notes-Repo-Copy.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/ML_super_cheatsheet.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/UPenn-CIS520-Final2017-Solutions.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/UPenn-CIS520-Final2018-Solutions.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/UPenn-CIS520-Midterm2022-Solutions.pdf
/kaggle/input/gate-dsai-llm/Machine-Learning/UPenn-CIS520-Midter

 =============================================================================
 STEP 1: ENVIRONMENT SETUP AND LIBRARY INSTALLATION
 =============================================================================

 In this step, we install all the necessary Python packages. Kaggle environments
 are ephemeral, so these installations are required for each session. We use
 quiet flags (-q) to keep the output clean.

 Key Libraries:
 - langchain: The core framework for building LLM applications.
 - transformers: Provides access to Hugging Face models and pipelines.
 - faiss-cpu: A library for efficient similarity search (our vector store).
 - sentence-transformers: Used for generating high-quality text embeddings.
 - pypdf: A robust library for extracting text from PDF documents.
 - accelerate: Required for efficiently loading and running large models on GPUs.

 =============================================================================

In [2]:
print("STEP 1: Installing required libraries...")
!pip install -q langchain "langchain-community[pdf]"
!pip install -q faiss-cpu sentence-transformers
!pip install -q transformers torch accelerate
print("Library installation complete.\n")

STEP 1: Installing required libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires matplotlib>=

 =============================================================================
 STEP 2: DATA LOADING AND DOCUMENT PROCESSING
 =============================================================================

 Here, we define the path to our custom dataset on Kaggle. We then
 recursively walk through the directory, identify all PDF files, and use
 LangChain's PyPDFLoader to load and split the documents into manageable
 chunks. This chunking is essential for the retrieval process.

 =============================================================================

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader

docs_path = "/kaggle/input/gate-dsai-llm"

print(f"STEP 2: Loading documents from '{docs_path}'...")

all_docs = []
# os.walk is a Python generator that explores a directory tree.
for root, _, files in os.walk(docs_path):
    for file in files:
        if file.endswith(".pdf"):
            try:
                file_path = os.path.join(root, file)
                loader = PyPDFLoader(file_path)
                # load_and_split() is more efficient than loading then splitting.
                loaded_docs = loader.load_and_split()
                all_docs.extend(loaded_docs)
                print(f"  -> Successfully loaded and split: {file}")
            except Exception as e:
                print(f"  -> ERROR loading {file}: {e}")

if not all_docs:
    raise ValueError("No documents were loaded. Check the dataset path and ensure it contains PDF files.")

print(f"\nTotal document chunks loaded: {len(all_docs)}\n")

STEP 2: Loading documents from '/kaggle/input/gate-dsai-llm'...
  -> Successfully loaded and split: GATE2024_DA_Sample_Paper.pdf
  -> Successfully loaded and split: GATE_DA_2025_Question_Paper.pdf
  -> Successfully loaded and split: GATE_DA_2025_Syllabus.pdf
  -> Successfully loaded and split: ML-A-Probabilistic-Perspective-Murphy.pdf
  -> Successfully loaded and split: UPenn-CIS520-Midterm2019-Solutions.pdf
  -> Successfully loaded and split: CMU-ML-Notes.pdf
  -> Successfully loaded and split: CS229-Stanford-Lecture-Notes-Repo-Copy.pdf
  -> Successfully loaded and split: ML_super_cheatsheet.pdf
  -> Successfully loaded and split: UPenn-CIS520-Final2017-Solutions.pdf
  -> Successfully loaded and split: UPenn-CIS520-Final2018-Solutions.pdf
  -> Successfully loaded and split: UPenn-CIS520-Midterm2022-Solutions.pdf
  -> Successfully loaded and split: UPenn-CIS520-Midterm2018-Solutions.pdf
  -> Successfully loaded and split: CS229-Stanford-Lecture-Notes.pdf
  -> Successfully loaded and sp

 =============================================================================
 STEP 3: VECTOR EMBEDDING AND INDEX CREATION
 =============================================================================

 This is the core of the "Retrieval" part of RAG. We convert our text chunks
 into numerical vectors (embeddings) using a sentence-transformer model.
 These vectors are then stored in a FAISS index, which allows for extremely
 fast similarity searches. When a user asks a question, we will embed their
 question and use this index to find the most semantically relevant chunks.

 =============================================================================

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

print("STEP 3: Creating vector embeddings and FAISS index...")

# Initialize the embedding model. 'all-MiniLM-L6-v2' is a great, lightweight
# model that provides excellent performance for semantic search.
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Create the FAISS vector store from our document chunks and embeddings.
# This process can take a few minutes depending on the number of documents.
vector_store = FAISS.from_documents(all_docs, embeddings)

print("Vector index created successfully.\n")
output_path = "/kaggle/working/my_vector_index"
vector_store.save_local(output_path)

print(f"STEP 4: Index successfully saved to '{output_path}'")
print("\n--- BUILD PROCESS COMPLETE ---")
print("You can now run the 'chatbot.ipynb' notebook.")

STEP 3: Creating vector embeddings and FAISS index...


/tmp/ipykernel_36/3473037497.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
2025-08-01 10:37:42.650595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754044663.008009      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754044663.096525      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector index created successfully.

STEP 4: Index successfully saved to '/kaggle/working/my_vector_index'

--- BUILD PROCESS COMPLETE ---
You can now run the 'chatbot.ipynb' notebook.


 =============================================================================
 STEP 4: LARGE LANGUAGE MODEL (LLM) INITIALIZATION
 =============================================================================

 We now load our generative AI model. We are using Microsoft's Phi-3-mini,
 a state-of-the-art small language model known for high-quality reasoning
 and generation. We use the 'transformers' library to create a pipeline,
 which is then wrapped by LangChain for seamless integration.

 `device_map="auto"` intelligently utilizes the available Kaggle GPU.

=============================================================================

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

print("STEP 4: Initializing the Large Language Model (Phi-3-mini)...")
print("(This may take a few minutes as the model is loaded into the GPU)")

# Define the model ID from Hugging Face
model_id = "microsoft/Phi-3-mini-4k-instruct"

# Load the model and its tokenizer.
# `trust_remote_code=True` is required for this model architecture.
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto" # This automatically uses the GPU
)

# Create a text-generation pipeline.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Max length of the generated answer
    return_full_text=False # We only want the generated response
)

# Wrap the pipeline in a LangChain-compatible object.
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM initialized successfully.\n")

 =============================================================================
 STEP 5: CREATING THE QUESTION-ANSWERING (QA) CHAIN
 =============================================================================

 This step ties everything together. We create a `RetrievalQA` chain, which
 orchestrates the entire RAG process. We use the "refine" chain type, which
 is robust for detailed questions. It iterates through the retrieved documents,
 progressively building and refining the answer, which helps generate more
 comprehensive and accurate responses.

 =============================================================================

In [ ]:
from langchain.chains import RetrievalQA

print("STEP 5: Creating the RetrievalQA chain...")

# The "refine" chain type is ideal for generating detailed answers from
# multiple document sources.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
)

print("QA chain is ready.\n")


 =============================================================================
 STEP 6: INTERACTIVE CHATBOT INTERFACE
 =============================================================================

 This final block runs a simple command-line interface, allowing you to
 interact with your custom GATE DSAI study bot.

 =============================================================================

In [ ]:
print("--- GATE DSAI Study Bot Initialized ---")
print("You can now ask questions about your documents.")
print("Type 'quit' or 'exit' to end the session.")

while True:
    try:
        query = input("\nYour Question: ")
        if query.lower() in ["quit", "exit"]:
            print("Session ended. Goodbye!")
            break
        if not query.strip():
            continue

        # Invoke the QA chain with the user's query
        result = qa_chain.invoke({"query": query})

        # Display the results
        print("\nAnswer:")
        print(result["result"])
        print("\nSources Used:")
        # Display the source documents for verification
        for doc in result["source_documents"]:
            print(f"  - {doc.metadata.get('source', 'Unknown source')}")

    except Exception as e:
        print(f"\nAn error occurred: {e}")